# 👥 Face Recognition Praktika - dlib eta face_recognition

**Egilea**: Mikel Aldalur Corta  
**Data**: 2025  
**Maila**: Aurreratua  

---

## 🎯 Helburua

Notebook honetan **Face Recognition praktika** egingo dugu Python liburutegiak erabiliz:
- face_recognition liburutegia (dlib-en wrapper)
- Aurpegiak detektatu
- Face encodings sortu
- Aurpegiak konparatu eta identifikatu
- Webcam-etik real-time recognition

---

## 📚 Ikasiko duguna

1. ✅ face_recognition liburutegia instalatu eta erabili
2. ✅ Aurpegiak detektatu irudian
3. ✅ Face landmarks aurkitu
4. ✅ Face encodings (128D) sortu
5. ✅ Aurpegiak konparatu eta identifikatu
6. ✅ Real-time recognition (kontzeptua)

## 1️⃣ Instalazioa

### 📦 Behar diren liburutegiak

```bash
# dlib instalatu (beharrezkoa)
pip install dlib

# face_recognition instalatu
pip install face_recognition

# OpenCV (bideoak prozesatzeko)
pip install opencv-python

# Pillow (irudiak prozesatzeko)
pip install Pillow
```

### ⚠️ Oharra

- **Windows**: dlib instalatzea korapilatsua izan daiteke. CMake eta Visual Studio C++ behar ditu.
- **Linux/Mac**: Errazagoa da.
- **Alternatiba**: Docker erabiliz edo Google Colab-en exekutatu.

## 2️⃣ Liburutegiak Inportatu

In [ ]:
# Liburutegiak
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from PIL import Image, ImageDraw, ImageFont
import warnings
warnings.filterwarnings('ignore')

# Estiloa
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
np.random.seed(42)

print("✅ Liburutegiak oinarrizkoak kargatuta!")

# face_recognition probatu
try:
    import face_recognition
    print("✅ face_recognition liburutegia erabilgarri!")
    print(f"   Bertsioa: {face_recognition.__version__ if hasattr(face_recognition, '__version__') else 'Ezezaguna'}")
except ImportError:
    print("⚠️ face_recognition ez dago instalatuta.")
    print("   Instala ezazu: pip install face_recognition")
    face_recognition = None

## 3️⃣ Face Detection Simulazioa

In [ ]:
# Face detection simulazioa
def simulatu_face_detection():
    """
    Face detection emaitza simulatu
    """
    # Irudi simulatua sortu
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    
    img = np.random.rand(500, 600, 3) * 0.3 + 0.4
    ax.imshow(img)
    
    # Aurpegi-kokalekuak simulatu
    aurpegiak = [
        (100, 100, 150, 200),  # (top, right, bottom, left)
        (300, 150, 400, 250),
        (150, 400, 250, 500)
    ]
    
    for i, (top, right, bottom, left) in enumerate(aurpegiak, 1):
        # Bounding box marraztu
        width = right - left
        height = bottom - top
        rect = patches.Rectangle(
            (left, top), width, height,
            linewidth=2, edgecolor='lime', facecolor='none'
        )
        ax.add_patch(rect)
        
        # Label
        ax.text(left, top-10, f'Aurpegia {i}',
               bbox=dict(facecolor='lime', alpha=0.7),
               fontsize=11, color='black', fontweight='bold')
    
    ax.set_title('👤 Face Detection Simulazioa', 
                fontsize=16, fontweight='bold', pad=20)
    ax.axis('off')
    plt.tight_layout()
    plt.show()
    
    print(f"✅ {len(aurpegiak)} aurpegi detektatuta!")

simulatu_face_detection()

## 4️⃣ Face Detection Kodea (Erreala)

```python
import face_recognition

# Irudia kargatu
image = face_recognition.load_image_file("path/to/image.jpg")

# Aurpegiak detektatu
face_locations = face_recognition.face_locations(image)

print(f"Detektatutako aurpegiak: {len(face_locations)}")

# Bakoitza marraztu
for (top, right, bottom, left) in face_locations:
    print(f"Aurpegi bat: Top={top}, Right={right}, Bottom={bottom}, Left={left}")
```

### 📊 Detekzio Metodoak

```python
# HOG metodoa (azkarragoa, defektua)
face_locations = face_recognition.face_locations(image, model="hog")

# CNN metodoa (zehatza, motela, GPU behar)
face_locations = face_recognition.face_locations(image, model="cnn")
```

## 5️⃣ Face Landmarks

**Face Landmarks**: Aurpegiaren puntu garrantzitsuak (begiak, sudurra, ahoa, etab.)

In [ ]:
# Face landmarks simulazioa
def simulatu_face_landmarks():
    """
    Face landmarks simulazioa
    """
    fig, ax = plt.subplots(1, 1, figsize=(8, 10))
    
    # Aurpegi-forma oinarrizkoa
    img = np.ones((400, 300, 3)) * 0.9
    ax.imshow(img)
    
    # Landmark puntuak simulatu
    landmarks = {
        'chin': [(150 + i*3, 50 + i*20) for i in range(17)],
        'left_eyebrow': [(80 + i*10, 120 - i*2) for i in range(5)],
        'right_eyebrow': [(170 + i*10, 120 - (4-i)*2) for i in range(5)],
        'left_eye': [(90, 150), (100, 145), (110, 145), (120, 150), (110, 155), (100, 155)],
        'right_eye': [(180, 150), (190, 145), (200, 145), (210, 150), (200, 155), (190, 155)],
        'nose': [(150, 180), (145, 200), (150, 210), (155, 200)],
        'top_lip': [(110, 250), (130, 245), (150, 240), (170, 245), (190, 250)],
        'bottom_lip': [(190, 250), (170, 260), (150, 265), (130, 260), (110, 250)]
    }
    
    koloreak = {
        'chin': 'blue',
        'left_eyebrow': 'red',
        'right_eyebrow': 'red',
        'left_eye': 'green',
        'right_eye': 'green',
        'nose': 'orange',
        'top_lip': 'purple',
        'bottom_lip': 'purple'
    }
    
    for part_name, points in landmarks.items():
        xs = [p[0] for p in points]
        ys = [p[1] for p in points]
        ax.plot(xs, ys, 'o-', color=koloreak[part_name], 
               linewidth=2, markersize=6, label=part_name)
    
    ax.set_title('🎯 Face Landmarks (68 puntos)', 
                fontsize=16, fontweight='bold', pad=20)
    ax.legend(loc='upper right', fontsize=9)
    ax.axis('off')
    ax.set_xlim([50, 250])
    ax.set_ylim([350, 50])
    plt.tight_layout()
    plt.show()

simulatu_face_landmarks()
print("✅ Face landmarks (68 puntu) bistaratuta!")

## 6️⃣ Face Landmarks Kodea (Erreala)

```python
import face_recognition

# Irudia kargatu
image = face_recognition.load_image_file("path/to/image.jpg")

# Face landmarks lortu
face_landmarks_list = face_recognition.face_landmarks(image)

for face_landmarks in face_landmarks_list:
    for facial_feature in face_landmarks.keys():
        print(f"{facial_feature}: {len(face_landmarks[facial_feature])} puntos")
```

### 📊 Landmark Aplikazioak

- **Face Alignment**: Aurpegiak lerrokatu
- **Emotion Recognition**: Emozioak detektatu
- **Face Filters**: Snapchat-en antzeko filtroak
- **Makeup**: Makillaje birtuala aplikatu

## 7️⃣ Face Encodings (128D)

Face encoding-a **128 dimensioko bektore** bat da aurpegiaren ezaugarriak adierazten dituena.

In [ ]:
# Face encodings simulazioa
def simulatu_face_encodings():
    """
    Face encodings simulazioa
    """
    # 3 pertsonaren encodings simulatuak
    mikel_encoding = np.random.randn(128)
    ane_encoding = np.random.randn(128)
    jon_encoding = np.random.randn(128)
    
    # Mikel-en beste argazki bat (antzekoa izan behar du)
    mikel_encoding_2 = mikel_encoding + np.random.randn(128) * 0.1
    
    print("🧠 Face Encodings (128D):")
    print("="*60)
    print(f"Mikel encoding:   [{mikel_encoding[0]:.3f}, {mikel_encoding[1]:.3f}, {mikel_encoding[2]:.3f}, ...]")
    print(f"Ane encoding:     [{ane_encoding[0]:.3f}, {ane_encoding[1]:.3f}, {ane_encoding[2]:.3f}, ...]")
    print(f"Jon encoding:     [{jon_encoding[0]:.3f}, {jon_encoding[1]:.3f}, {jon_encoding[2]:.3f}, ...]")
    print(f"Mikel encoding 2: [{mikel_encoding_2[0]:.3f}, {mikel_encoding_2[1]:.3f}, {mikel_encoding_2[2]:.3f}, ...]")
    print("="*60)
    
    # Distantziak kalkulatu
    dist_mikel_mikel = np.linalg.norm(mikel_encoding - mikel_encoding_2)
    dist_mikel_ane = np.linalg.norm(mikel_encoding - ane_encoding)
    dist_mikel_jon = np.linalg.norm(mikel_encoding - jon_encoding)
    
    print("\n📏 Distantziak:")
    print(f"  Mikel vs Mikel (aurpegi bera): {dist_mikel_mikel:.4f}")
    print(f"  Mikel vs Ane (desberdina):     {dist_mikel_ane:.4f}")
    print(f"  Mikel vs Jon (desberdina):     {dist_mikel_jon:.4f}")
    
    # Bistaratu
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    
    labels = ['Mikel vs Mikel\n(aurpegi bera)', 'Mikel vs Ane\n(desberdina)', 'Mikel vs Jon\n(desberdina)']
    distantziak = [dist_mikel_mikel, dist_mikel_ane, dist_mikel_jon]
    colors = ['green', 'red', 'red']
    
    bars = ax.bar(labels, distantziak, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
    ax.axhline(y=0.6, color='blue', linestyle='--', linewidth=2, label='Threshold (0.6)')
    ax.set_ylabel('Distantzia Euklidarra', fontsize=12)
    ax.set_title('📏 Face Encodings - Distantzien Konparaketa', 
                fontsize=16, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3, axis='y')
    
    # Balioak gehitu
    for bar, dist in zip(bars, distantziak):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{dist:.4f}',
               ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

simulatu_face_encodings()
print("\n✅ Face encodings simulazioa osatuta!")

## 8️⃣ Face Encodings Kodea (Erreala)

```python
import face_recognition

# Irudiak kargatu
mikel_image = face_recognition.load_image_file("mikel.jpg")
ane_image = face_recognition.load_image_file("ane.jpg")

# Encodings sortu
mikel_encoding = face_recognition.face_encodings(mikel_image)[0]
ane_encoding = face_recognition.face_encodings(ane_image)[0]

print(f"Mikel encoding: {mikel_encoding.shape}")  # (128,)
print(f"Ane encoding: {ane_encoding.shape}")      # (128,)
```

## 9️⃣ Face Comparison eta Recognition

In [ ]:
# Face comparison simulazioa
def simulatu_face_comparison():
    """
    Face comparison eta recognition simulazioa
    """
    # Datu-baseko encodings
    database = {
        'Mikel': np.random.randn(128),
        'Ane': np.random.randn(128),
        'Jon': np.random.randn(128),
        'Leire': np.random.randn(128)
    }
    
    # Test encodings
    test_cases = [
        ('Mikel argazki berria', database['Mikel'] + np.random.randn(128) * 0.1, 'Mikel'),
        ('Ane argazki berria', database['Ane'] + np.random.randn(128) * 0.15, 'Ane'),
        ('Ezezagun bat', np.random.randn(128), 'Ezezaguna')
    ]
    
    threshold = 0.6
    
    print("🔍 Face Recognition Simulazioa:")
    print("="*70)
    
    for test_name, test_encoding, expected in test_cases:
        print(f"\n📸 Test: {test_name}")
        print(f"   Espero: {expected}")
        
        # Datu-baseko pertsona guztiekin konparatu
        distantziak = {}
        for izena, encoding in database.items():
            dist = np.linalg.norm(test_encoding - encoding)
            distantziak[izena] = dist
        
        # Hurbilena aurkitu
        hurbilena = min(distantziak, key=distantziak.get)
        dist_min = distantziak[hurbilena]
        
        # Identifikatu
        if dist_min < threshold:
            identifikatua = hurbilena
            status = "✅"
        else:
            identifikatua = "Ezezaguna"
            status = "❌"
        
        print(f"   Emaitza: {identifikatua} (distantzia: {dist_min:.4f})")
        print(f"   {status} {'Zuzena!' if identifikatua == expected else 'Okerra!'}")
        
        # Distantzia guztiak
        print(f"   Distantziak:")
        for izena, dist in sorted(distantziak.items(), key=lambda x: x[1]):
            print(f"      - {izena}: {dist:.4f}")
    
    print("\n" + "="*70)

simulatu_face_comparison()
print("✅ Face comparison simulazioa osatuta!")

## 🔟 Face Recognition Kodea Osoa (Erreala)

```python
import face_recognition
import numpy as np

# 1. Datu-basea sortu (ezagunak)
mikel_image = face_recognition.load_image_file("mikel.jpg")
ane_image = face_recognition.load_image_file("ane.jpg")

mikel_encoding = face_recognition.face_encodings(mikel_image)[0]
ane_encoding = face_recognition.face_encodings(ane_image)[0]

known_encodings = [mikel_encoding, ane_encoding]
known_names = ["Mikel", "Ane"]

# 2. Test irudia
test_image = face_recognition.load_image_file("test.jpg")
test_encoding = face_recognition.face_encodings(test_image)[0]

# 3. Konparatu
matches = face_recognition.compare_faces(known_encodings, test_encoding)
name = "Ezezaguna"

# Distantziak kalkulatu
face_distances = face_recognition.face_distance(known_encodings, test_encoding)
best_match_index = np.argmin(face_distances)

if matches[best_match_index]:
    name = known_names[best_match_index]

print(f"Aurpegia identifikatuta: {name}")
print(f"Distantzia: {face_distances[best_match_index]:.4f}")
```

## 1️⃣1️⃣ Webcam Real-Time Recognition (Kontzeptua)

```python
import face_recognition
import cv2

# Webcam ireki
video_capture = cv2.VideoCapture(0)

# Datu-basea kargatu
known_encodings = [mikel_encoding, ane_encoding]
known_names = ["Mikel", "Ane"]

while True:
    # Frame bat irakurri
    ret, frame = video_capture.read()
    
    # BGR -> RGB
    rgb_frame = frame[:, :, ::-1]
    
    # Aurpegiak detektatu eta encodings lortu
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
    
    # Aurpegi bakoitza identifikatu
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(known_encodings, face_encoding)
        name = "Ezezaguna"
        
        face_distances = face_recognition.face_distance(known_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        
        if matches[best_match_index]:
            name = known_names[best_match_index]
        
        # Marraztu
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                   0.75, (0, 255, 0), 2)
    
    # Bistaratu
    cv2.imshow('Face Recognition', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()
```

## 1️⃣2️⃣ Performance Optimizazioak

### ⚡ Optimizazio Aholkuak

1. **Frame-ak saltatu**: Ez prozesatu frame guztiak
```python
frame_count = 0
process_every = 5  # Prozesatu 5. frame-a bakarrik

if frame_count % process_every == 0:
    # Prozesatu frame-a
    pass
frame_count += 1
```

2. **Tamaina txikitu**: Frame-ak txikitu prozesatzeko
```python
small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
```

3. **HOG vs CNN**: HOG azkarra, CNN zehatza
```python
# Azkarragoa
face_locations = face_recognition.face_locations(frame, model="hog")

# Zehatza baina motela
face_locations = face_recognition.face_locations(frame, model="cnn")
```

4. **GPU**: dlib-ek GPU euskarria du
```python
import dlib
dlib.DLIB_USE_CUDA = True
```

In [ ]:
# Performance konparaketa simulazioa
def simulatu_performance():
    """
    Metodo desberdinen performance simulazioa
    """
    metodoak = ['HOG (CPU)', 'CNN (CPU)', 'CNN (GPU)']
    fps = [30, 5, 25]
    zehaztasuna = [0.95, 0.99, 0.99]
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    fig.suptitle('📊 Face Recognition Performance Konparaketa', 
                 fontsize=16, fontweight='bold')
    
    # FPS
    colors_fps = ['green', 'red', 'orange']
    bars1 = axes[0].bar(metodoak, fps, color=colors_fps, alpha=0.7, 
                       edgecolor='black', linewidth=2)
    axes[0].set_ylabel('FPS (Frames per second)', fontsize=12)
    axes[0].set_title('⚡ Abiadura', fontsize=14, fontweight='bold')
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # Balioak
    for bar, val in zip(bars1, fps):
        height = bar.get_height()
        axes[0].text(bar.get_x() + bar.get_width()/2., height,
                   f'{val} FPS',
                   ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    # Zehaztasuna
    colors_acc = ['orange', 'green', 'green']
    bars2 = axes[1].bar(metodoak, zehaztasuna, color=colors_acc, alpha=0.7, 
                       edgecolor='black', linewidth=2)
    axes[1].set_ylabel('Zehaztasuna', fontsize=12)
    axes[1].set_title('🎯 Zehaztasuna', fontsize=14, fontweight='bold')
    axes[1].set_ylim([0.9, 1.0])
    axes[1].grid(True, alpha=0.3, axis='y')
    
    # Balioak
    for bar, val in zip(bars2, zehaztasuna):
        height = bar.get_height()
        axes[1].text(bar.get_x() + bar.get_width()/2., height,
                   f'{val:.2%}',
                   ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    for ax in axes:
        ax.tick_params(axis='x', rotation=15)
    
    plt.tight_layout()
    plt.show()

simulatu_performance()
print("✅ Performance konparaketa bistaratuta!")

## 📝 Ondorioak

### 🎯 Ikasitako Gauzak

1. ✅ face_recognition liburutegia erabili
2. ✅ Aurpegiak detektatu (HOG, CNN)
3. ✅ Face landmarks (68 puntu) aurkitu
4. ✅ Face encodings (128D) sortu
5. ✅ Aurpegiak konparatu eta identifikatu
6. ✅ Real-time recognition implementatu (kontzeptua)

### 📊 face_recognition Liburutegiaren Abantailak

- ✅ **Erraza**: API oso sinplea
- ✅ **Zehatza**: dlib-en gainean eraikita (99.38% zehaztasuna)
- ✅ **Osoa**: Detection, landmarks, encoding, comparison
- ✅ **Dokumentazioa**: Oso ondo dokumentatuta

### 🔑 Aplikazio Erabilgarriak

1. **Segurtasuna**: Sarrera-kontrola, autentifikazioa
2. **Asistentzia**: Langile/ikasle asistentzia kontrola
3. **Sare sozialak**: Auto-tagging (argazkietan pertsonak etiketatu)
4. **Smart Home**: Etxeko automatizazioa
5. **Bilaketa**: Argazki datu-basean pertsonak bilatu

### ⚠️ Etikako Kontuak

- **Pribatutasuna**: Datu pertsonalak babestea (GDPR)
- **Baimena**: Beti eskatu baimena
- **Bias**: Arrazaren, generoaren, adinaren sesgoak
- **Transparentzia**: Erabiltzaileei jakinarazi
- **Erabilera**: Ez diskriminazio-erabileretarako

### 🚀 Hurrengo Pausoak

- **Custom dataset**: Gure pertsonak identifikatzeko sistema
- **Database**: SQL/MongoDB datu-basea integratu
- **Web app**: Flask/FastAPI aplikazio bat sortu
- **Mobile**: React Native edo Flutter app-a
- **Cloud**: AWS Rekognition, Azure Face API

---

**Egilea**: Mikel Aldalur Corta  
**Data**: 2025  
**Lizentzia**: MIT